## Step 1: Hello, Data!

In this step, I load the raw sales transactions CSV file into a DataFrame and display the first three rows to understand the structure of the data.


In [1]:
# ---------- importing the pandas library ----------
import pandas as pd

# ---------- reading  CSV file ----------
file1 = r"D:\Conestoga\Machine Learning Programming\Lab-2\sales_data_with_coupons.csv"
data1 = pd.read_csv(file1)

# ---------- displaying the first 3 rows of the data ----------
print(data1.head(3))



   sno                            Region Country   Item Type Sales Channel  \
0    1      middle east and north africa   Libya   Cosmetics       Offline   
1    2                     north america  CANADA  Vegetables        Online   
2    3    Middle East and North Africa     LIBYA   Baby Food       Offline   

  Order Priority  Order Date   Order ID   Ship Date  Units Sold  Unit Price  \
0              M  2014-10-18  686800706  31-10-2014        8446      437.20   
1              M  07-11-2011  185941302  2011-12-08        3018      154.06   
2              C  2016-10-31  246222341  2016-12-09        1517      255.28   

   Unit Cost  Total Revenue  Total Cost  Total Profit coupon_code  
0     263.33     3692591.20  2224085.18    1468506.02      GF24TA  
1      90.93      464953.08   274426.74     190526.34      10AMSP  
2     159.42      387259.76   241840.14     145419.62      TEYPEU  


## Step 2: Pick the Right Container

A `dict` is simple but doesn’t group behavior with data and a `namedtuple` is cleaner than a dict,but no custom functions
A `class` is best here  because it helps organize the data and lets me add my own methods like `clean()` and `total()`.

## Step 3: Transaction Class and OO structure 


In [2]:
# ---------- creating a class to represent one transaction ----------

class Transaction:
    def __init__(self, row):
        # ---------- saving each column into the object ----------
        self.order_date = row["Order Date"]
        self.customer_id = row["Order ID"]
        self.product = row["Item Type"]
        self.price = row["Unit Price"]
        self.quantity = row["Units Sold"]
        self.coupon_code = row["coupon_code"]
        self.shipping_city = row["Country"] 

# ---------- creating a list to store all transaction objects ----------
transaction_list = []

# ---------- going through each row and converting to a Transaction object ----------
for index, row in data1.iterrows():
    obj = Transaction(row)
    transaction_list.append(obj)

# ---------- printing the first transaction as a check ----------
print(vars(transaction_list[0]))


{'order_date': '2014-10-18', 'customer_id': 686800706, 'product': 'Cosmetics', 'price': 437.2, 'quantity': 8446, 'coupon_code': 'GF24TA', 'shipping_city': 'Libya'}



In this step i created a Python class to represent each row in the sales data.
I used real column names like "Order Date", "Item Type", and "coupon_code" from the CSV.


##  Step 4: Bulk Loader 


In [3]:
# ---------- importing List for return type hint ----------
from typing import List

# ---------- function to load all transactions ----------

def load_transactions(dataframe) -> List[Transaction]:
    # ---------- list to hold transaction objects ----------
    result = []

    # ---------- loop  each row in the dataframe ----------
    for index, row in dataframe.iterrows():
        obj = Transaction(row)      
        result.append(obj)          

    return result

# ---------- using the function to load data ----------
transactions = load_transactions(data1)

print(vars(transactions[0]))


{'order_date': '2014-10-18', 'customer_id': 686800706, 'product': 'Cosmetics', 'price': 437.2, 'quantity': 8446, 'coupon_code': 'GF24TA', 'shipping_city': 'Libya'}


in this step i made a function called `load_transactions()` to turn each row of the data into a `Transaction` object.
This function goes through the dataframe row by row and makes a list of all  transactions.
It helps to keep the code clean and now i can just call this function anytime I want to load the data as objects.


##  Step 5: Quick Profiling 


In [4]:
# ---------- getting min, mean, and max of unit price ----------

min_price = data1["Unit Price"].min()
mean_price = data1["Unit Price"].mean()
max_price = data1["Unit Price"].max()

print("Min price:", min_price)
print("Mean price:", mean_price)
print("Max price:", max_price)

# ---------- getting the number of unique shipping cities ----------
unique_cities = data1["Country"].nunique()

print("Number of unique shipping cities:", unique_cities)


Min price: 9.33
Mean price: 274.29506
Max price: 668.27
Number of unique shipping cities: 360


so calculated the minimum, average, and maximum values for the "Unit Price" column using the pandas functions `.min()`, `.mean()`,  `.max()` and also counted how many cities are in the data. It showed 360, but this is wrong.

Some city names are written in different ways, like "canada" and "CANADA", so they are counted more than onceI will fix this in the next steps.


##  Step 6: Spot the Grime  

In [5]:

# ---------- Missing values ----------
print("Missing values in each column:")
print(data1.isnull().sum())

# ---------- Duplicate rows ----------
print("Number of duplicate rows:")
print(data1.duplicated().sum())

# ----------  Negative prices ----------
print("Negative Unit Prices:")
print((data1["Unit Price"] < 0).sum())

# ----------  Zero or negative quantities ----------
print("Zero or negative Units Sold:")
print((data1["Units Sold"] <= 0).sum())

# ----------  Country names not standardized ----------
print("Different countries before cleaning:")
print(data1["Country"].nunique())

# ---------- check how many unique countries after standardizing----------
fixed_countries = data1["Country"].str.strip().str.lower()
print("Different countries after cleaning:")
print(fixed_countries.nunique())

# ----------   check date columns ----------
data1["Order Date"] = pd.to_datetime(data1["Order Date"], errors='coerce', dayfirst=True)

print("Bad Order Dates:", data1["Order Date"].isna().sum())

Missing values in each column:
sno               0
Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
coupon_code       0
dtype: int64
Number of duplicate rows:
0
Negative Unit Prices:
0
Zero or negative Units Sold:
0
Different countries before cleaning:
360
Different countries after cleaning:
171
Bad Order Dates: 182


C:\Users\kittu\AppData\Local\Temp\ipykernel_24644\1766649505.py:27: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data1["Order Date"] = pd.to_datetime(data1["Order Date"], errors='coerce', dayfirst=True)


## Spot the Grime  
country  names have issues  
1. they have extra spaces 
2. different cases. 
3. Dates were not in the same format.


##  Step 7: Cleaning Rules 


In [6]:

# ------------------to Convert Order Date -------------------- 
data1["Order Date"] = pd.to_datetime(data1["Order Date"], errors="coerce", dayfirst=True)

# ----------  Transaction class with clean() ----------
class Transaction:
    def __init__(self, row):
        self.order_date = row["Order Date"]
        self.customer_id = row["Order ID"]
        self.product = row["Item Type"]
        self.price = row["Unit Price"]
        self.quantity = row["Units Sold"]
        self.coupon_code = row["coupon_code"]
        self.shipping_city = row["Country"]

    def clean(self):
        # ---------- Clean city ----------
        if pd.notnull(self.shipping_city):
            self.shipping_city = str(self.shipping_city).strip().lower()
        
        # ---------- Clean coupon ----------
        if pd.notnull(self.coupon_code):
            self.coupon_code = str(self.coupon_code).strip().upper()
        
        
        # ---------- Re-parse order date ----------
        try:
            self.order_date = pd.to_datetime(self.order_date, errors="coerce", dayfirst=True)
        except:
            self.order_date = pd.NaT

# ----------  Load and clean transactions ----------
def load_transactions(df):
    return [Transaction(row) for index, row in df.iterrows()]

transactions = load_transactions(data1)

# ---------- Show before cleaning ----------
before_cities = len(set(t.shipping_city for t in transactions if pd.notnull(t.shipping_city)))

# ---------- Apply cleaning ----------
for t in transactions:
    t.clean()

# ---------- Show after cleaning ----------
after_cities = len(set(t.shipping_city for t in transactions if pd.notnull(t.shipping_city)))
# ---------- remove any transactions with missing order_date ----------
transactions = [t for t in transactions if pd.notnull(t.order_date)]


print("Cities before cleaning:", before_cities)
print("Cities after cleaning:", after_cities)
print("Order dates cleaned ")


Cities before cleaning: 360
Cities after cleaning: 171
Order dates cleaned 


In this step, I cleaned up two main fields: the country names and the order dates.

- For country names, I removed extra spaces and changed them all to lowercase so they are consistent.
- For order dates, I used pandas to convert all date formats into one clean datet

#  Step 8: Transformations

In [15]:

# ---------- load the second file that has coupon info ----------
coupon_info = pd.read_csv("coupon_metadata_full_randomized.csv")
print(coupon_info.head())

# ---------- make a dictionary from it ----------
coupon_dict = dict(zip(coupon_info["coupon_code"].str.upper(), coupon_info["discount_percent"]))

# ---------- update the clean() method to include discount ----------
for t in transactions:
    # ---------- make sure code is uppercase ----------
    if t.coupon_code:
        code = t.coupon_code.upper()
        t.discount_percent = coupon_dict.get(code, 0)
    else:
        t.discount_percent = 0


   sno coupon_code    coupon_name  discount_percent
0    1      8U2V5D  FREESHIP SALE                20
1    2      3MPVLN     FLASH SALE                30
2    3      8A99EW    SUMMER SALE                25
3    4      BG5G5Q   NEWUSER SALE                20
4    5      T7GK01    WINTER SALE                30



In this step, I used the `coupon_code` to add a new field called `discount_percent` and  loaded a second file which had the discounts for each code. Then I matched them using a dictionary.

If the code was missing or not in the list, I just used 0 as the discount.


## Step 9: Feature Engineering


In [8]:

import pandas as pd

# ---------- get today's date ----------
today = pd.Timestamp.today()

# ---------- calculate days since purchase for each transaction ----------
for t in transactions:
    if pd.notnull(t.order_date):
        diff = today - t.order_date
        t.days_since_purchase = diff.days
    else:
        t.days_since_purchase = None
# Print the first 5 transactions with their order date and days_since_purchase
sample_data = [
    {
        "Order Date": t.order_date,
        "Days Since Purchase": t.days_since_purchase
    }
    for t in transactions[:5]
]

import pandas as pd
pd.DataFrame(sample_data)


,Order Date,Days Since Purchase
0,2014-10-18,3875
1,2016-10-31,3131
2,2015-03-04,3738
3,2012-05-17,4759
4,2015-01-29,3772



Here I added a new column called `days_since_purchase`and used Python to check how many days ago each order happened by subtracting the order date from today’s date.

If the order date was missing I just left it empty (None).


## Step 10: Mini Aggregation


In [9]:

# ---------- make a dictionary to store city revenue ----------
city_revenue = {}

for t in transactions:
    if t.shipping_city:
        revenue = t.price * t.quantity
        if t.shipping_city in city_revenue:
            city_revenue[t.shipping_city] += revenue
        else:
            city_revenue[t.shipping_city] = revenue

for city, revenue in list(city_revenue.items())[:5]:
    print(city, "=", round(revenue, 2))


libya = 4384286.96
eritrea = 1237900.62
montenegro = 1514116.87
jamaica = 713942.03
fiji = 1639227.62


I created a small summary that shows how much revenue came from each city.

To do this, I used a Python dictionary. I looped through all the transactions and calculated revenue (price × quantity), then added it by city name.

This helps us see where the most orders came from.


##  Step 11: Serialization Checkpoint 


In [10]:

import pandas as pd

# ---------- make a list of dictionaries from each Transaction ----------
records = []
for t in transactions:
    records.append({
        "order_date": t.order_date,
        "customer_id": t.customer_id,
        "product": t.product,
        "price": t.price,
        "quantity": t.quantity,
        "coupon_code": t.coupon_code,
        "shipping_city": t.shipping_city,
        "discount_percent": getattr(t, "discount_percent", 0),
        "days_since_purchase": getattr(t, "days_since_purchase", None)
    })

# ---------- turn into a DataFrame ----------
df_final = pd.DataFrame(records)

# ---------- save as JSON ----------
df_final.to_json("cleaned_data.json", orient="records", lines=True)

# ---------- save as Parquet ----------
df_final.to_parquet("cleaned_data.parquet", index=False)

print(" Data saved to JSON and Parquet!")


 Data saved to JSON and Parquet!


## Step 11: Saving the Data

I saved my final data into two files:

- One in JSON format
- One in Parquet format


## Step 12: Reflection – How OOP Helped

Objectoriented programming helped me keep my code simple and clean,i  created and used a class called `Transaction` to store each row of data. so i dont have to write code for dataframes everytime so i just call the calss also added a clean() function inside the class inside loop so i clean the data of each transaction simply and faster.
it saved time and kept my code organized. If I didn’t use OOP, I think the code would be much longer and harder to manage.


## Data Dictionary

| Field Name           | Type         | Description                        | Source                      |
|----------------------|--------------|------------------------------------|-----------------------------|
| order_date           | datetime     | Date when order was placed         | sales_data_with_coupons     |
| customer_id          | float        | ID of the customer                 | sales_data_with_coupons     |
| product              | string       | Name of the product                | sales_data_with_coupons     |
| price                | float        | Price per unit                     | sales_data_with_coupons     |
| quantity             | integer      | Number of units sold               | sales_data_with_coupons     |
| coupon_code          | string       | Discount code used                 | sales_data_with_coupons     |
| shipping_city        | string       | Name of the country (standardized) | sales_data_with_coupons     |
| discount_percent     | integer      | Discount percent from coupon       | coupon_metadata             |
| days_since_purchase  | integer      | Days since order was made          | calculated in notebook      |
